![](../img/330-banner.png)

Lecture 9: Classification Metrics
-----------

UBC 2022-23 W2

Instructor: Amir Abdi
 - Office Hours: Mondays 5-6 (or 5-7 if student turn-out was high)

<br><br><br>

## Announcements

- HW4 is due Feb 10, 11:59pm
- Midterm Feb 15 Wednesday
- Please make use of **OH** and **tutorials** to ask your questions.  

## Learning outcomes 

From this lecture, students are expected to be able to:

- Explain why accuracy is not always the best metric in ML.
- Explain components of a **confusion matrix**. 
- Define **precision**, **recall**, and **f1-score** and use them to evaluate different classifiers. 
- Broadly explain macro-average, weighted average.
- Interpret and use precision-recall curves. 
- Explain average precision score.
- Interpret and use ROC curves and ROC AUC using `scikit-learn`.  
- Identify whether there is class imbalance and whether you need to deal with it.
- Explain and use `class_weight` to deal with data imbalance.

## Legends

    
| <img src="https://upload.wikimedia.org/wikipedia/commons/f/f8/This_is_the_photo_of_Arthur_Samuel.jpg" width="100"> | <img src="http://www.cs.cmu.edu/~tom/TomHead2-6-22-22.jpg" width="100">  | <img src="https://upload.wikimedia.org/wikipedia/commons/4/49/John_McCarthy_Stanford.jpg" width="100"> | <img src="https://datascience.columbia.edu/wp-content/uploads/2020/08/Vapnik_web.png" width="100"> | <img src="https://upload.wikimedia.org/wikipedia/commons/a/a1/Alan_Turing_Aged_16.jpg" width="100"> | <img src="https://upload.wikimedia.org/wikipedia/commons/1/1e/Yoshua_Bengio_2019_cropped.jpg" width="100"> |
| :-----------: | :-----------: | :-----------: | :-----------: | :-----------: | :-----------: | 
| Arthur Samuel       | Tom Mitchell       |John McCarthy|  Vladimir N. Vapnik | Alan Turing | Yoshua Bengio |
| (1901-1990)    | 1951 - Now       |  1927 – 2011 | 1936 - Now | 1912 – 1954 | 1964-Now |
| First computer learning program | 1997 ML Texbook, CMU Prof | Co-coined term AI, Lisp,<br> Time-sharing, Garbage collection | SVM | Turing Test, Turning Machine | Turing Award<br> Father of Deep Learning


<br><br><br><br>

# Motivation (Evaluation metrics for binary classification)

In [ ]:
import os
import sys

sys.path.append("../code/.")

import IPython
import matplotlib.pyplot as plt
import mglearn
import numpy as np
import pandas as pd
from IPython.display import HTML, display
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, cross_validate, train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler

%matplotlib inline
pd.set_option("display.max_colwidth", 200)

from IPython.display import Image

# Changing global matplotlib settings for confusion matrix.
plt.rcParams["xtick.labelsize"] = 18
plt.rcParams["ytick.labelsize"] = 18


## Dataset for demonstration 

- Let's classify fraudulent and non-fraudulent transactions using Kaggle's [Credit Card Fraud Detection](https://www.kaggle.com/mlg-ulb/creditcardfraud) data set.

In [ ]:
cc_df = pd.read_csv("../data/creditcard.csv", encoding="latin-1")
train_df, test_df = train_test_split(cc_df, test_size=0.3, random_state=111)
train_df.head()

In [ ]:
train_df.shape

- Good size dataset (almost a **real-world** dataset!)
- For confidentially reasons, it only provides **transformed features with PCA** and **annonymized**, 
  - PCA is a popular dimensionality reduction technique [learn more here](https://en.wikipedia.org/wiki/Principal_component_analysis).

### EDA

In [ ]:
train_df.info()

In [ ]:
train_df.describe(include="all")

- We do not have categorical features. **All features are numeric**. 
- We have to be careful about the `Time` and `Amount` features. 
- We could scale `Amount`. 

- Do we want to scale **time**?
- What does **time** tell us about the sample?
  - Could some fradulent activities happen at the same time (temporal correlation)?
  - Could fradulent activities today tell us something about fradulent activities in the future?
  
For now, in this lecture, we will drop **time**; later on, in another lecture, we will focus more on **time** as a feature in time-series.

Let's separate `X` and `y` for train and test splits.

In [ ]:
X_train_big, y_train_big = train_df.drop(columns=["Class", "Time"]), train_df["Class"]
X_test, y_test = test_df.drop(columns=["Class", "Time"]), test_df["Class"]

- It's easier to demonstrate evaluation metrics using an **explicit validation** set instead of using cross-validation. 
- And we have **enough data**

<img src='../img/train-valid-test-split.png' width="1500" height="1500" />

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_big, y_train_big, test_size=0.3, random_state=123
)

### Baseline: DummyClassifier

In [ ]:
dummy = DummyClassifier()
dummy.fit(X_train, y_train)
dummy.score(X_valid, y_valid)

### Observations 

- `DummyClassifier` is getting **0.998 ACCURACY on validation set**!! 
- Should we be happy with this accuracy and deploy this `DummyClassifier` model for fraud detection? 

What's the class distribution? 

In [ ]:
train_df["Class"].value_counts()

- We have **class imbalance**. 
  - We have **MANY non-fraud transactions** and **only a handful of fraud transactions**. 
- So in the training set, `most_frequent` strategy is labeling 199,025 (99.83%) instances correctly and only 339 (0.17%) instances incorrectly. 
- Is this what we want? 
- The "fraud" class is the important class that we want to spot. 

-----------------
Let's scale the features and try `LogisticRegression`.   

In [ ]:
pipe = make_pipeline(StandardScaler(), LogisticRegression())
pipe.fit(X_train, y_train)
pipe.score(X_valid, y_valid)


- We are getting a slightly better score with logistic regression.  
- What score should be considered an acceptable score here? 
- Are we actually spotting any "fraud" transactions? 

- `.score` in **Classification** by default returns **ACCURACY** which is 
$$\text{accuracy} = \frac{\text{correct predictions}}{\text{total examples}}$$
- Is accuracy a good metric here? 
- Is there anything more informative than accuracy that we can use here? 

Let's dig a little deeper.

<br><br><br><br>

# Confusion matrix

One way to get a better understanding of the errors is by looking at 
- false positives (type I errors), where the model incorrectly spots examples as fraud
- false negatives (type II errors), where it's missing to spot fraud examples 

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

pipe.fit(X_train, y_train)

# --------- New Code ---------------
disp = ConfusionMatrixDisplay.from_estimator(
    pipe,
    X_valid,
    y_valid,
    display_labels=["Non fraud", "fraud"],
    values_format="d",
    cmap=plt.cm.Blues,
    colorbar=False,
);
# --------- New Code ---------------

Let's compare that with the Confusion Matrix of Dummy Clasifier:

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

pipe.fit(X_train, y_train)

disp = ConfusionMatrixDisplay.from_estimator(
    dummy,
    X_valid,
    y_valid,
    display_labels=["Non fraud", "fraud"],
    values_format="d",
    cmap=plt.cm.Blues,
    colorbar=False,
);

In [ ]:
from sklearn.metrics import confusion_matrix
from plotting_functions import plot_confusion_matrix_example

predictions = pipe.predict(X_valid)
TN, FP, FN, TP = confusion_matrix(y_valid, predictions).ravel()
plot_confusion_matrix_example(TN, FP, FN, TP)

- Perfect prediction has all values down the diagonal
- Off diagonal entries can often tell us about what is being mis-predicted

### What is "positive" and "negative"?

- Two kinds of binary classification problems 
    - Distinguishing between two classes
    - Spotting a class (spot fraud transaction, spot spam, spot disease)
- In case of spotting problems, the thing that we are interested in spotting is considered "positive". 
- Above we wanted to **spot (detect) fraudulent transactions** and so **Fraudulents are "positive"**. 

You can get a numpy array of confusion matrix as follows: 

In [ ]:
from sklearn.metrics import confusion_matrix

predictions = pipe.predict(X_valid)
TN, FP, FN, TP = confusion_matrix(y_valid, predictions).ravel()
print("Confusion matrix for fraud data set")
print(disp.confusion_matrix)

-----------
[Read for yourself at home

## Confusion matrix with cross-validation 

- You can also calculate confusion matrix with cross-validation using the `cross_val_predict` method.  
- But then you cannot conveniently use `plot_confusion_matrix`. 

In [ ]:
from sklearn.model_selection import cross_val_predict

confusion_matrix(y_train, cross_val_predict(pipe, X_train, y_train))

[end of Read for yourself at home

-----------


<br><br><br><br>

## Precision, recall, f1 score 

- We have been using `.score` to assess our models, which returns **ACCURACY** by default. 
- Accuracy is misleading when we have class imbalance.
- We need other metrics to assess our models.

- We'll discuss three commonly used metrics which are based on confusion matrix: 
    - recall
    - precision
    - f1 score 
- Later we'll talk about a few ways to address class imbalance problem. 

In [ ]:
from sklearn.metrics import confusion_matrix

pipe_lr = make_pipeline(StandardScaler(), LogisticRegression())
pipe_lr.fit(X_train, y_train)
predictions = pipe_lr.predict(X_valid)
TN, FP, FN, TP = confusion_matrix(y_valid, predictions).ravel()

Let's start with ACCURACY:

$$ accuracy = \frac{TP+TN}{all} = \frac{TP+TN}{TP+TN+FP+FN} $$

### Precision 

Among the positive examples you identified, how many were actually positive?

$$ precision = \frac{TP}{TP+FP} = \frac{TP}{\#detected\_positives} $$

------
Precision captures the **Quality** of detection
- from detected ones, how many are correct?

------

In [ ]:
ConfusionMatrixDisplay.from_estimator(
    pipe,
    X_valid,
    y_valid,
    display_labels=["Non fraud", "fraud"],
    values_format="d",
    cmap=plt.cm.Blues,
);

In [ ]:
print("TP = %0.4f, FP = %0.4f" % (TP, FP))
precision = TP / (TP + FP)
print("Precision: %0.4f" % (precision))

### Recall 

Among all positive examples, how many did you identify?
$$ recall = \frac{TP}{TP+FN} = \frac{TP}{\#actual\_positives} $$

------
Recall captures the **Quantity** of detecting positives
- How many of the total positives have we detected?

------

In [ ]:
ConfusionMatrixDisplay.from_estimator(
    pipe,
    X_valid,
    y_valid,
    display_labels=["Non fraud", "fraud"],
    values_format="d",
    cmap=plt.cm.Blues,
);

In [ ]:
print("TP = %0.4f, FN = %0.4f" % (TP, FN))
recall = TP / (TP + FN)
print("Recall: %0.4f" % (recall))

### F1-score

- F1-score combines precision and recall to give one score, which could be used in hyperparameter optimization, for instance. 
- F1-score is a **harmonic mean** of precision and recall.


$$ f1 = 2 \times \frac{ precision \times recall}{precision + recall}$$


In [ ]:
print("precision: %0.4f" % (precision))
print("recall: %0.4f" % (recall))
f1_score = (2 * precision * recall) / (precision + recall)
print("f1: %0.4f" % (f1_score))

Let's look at all metrics at once on our dataset.

In [ ]:
## Calculate evaluation metrics by ourselves
data = {
    "calculation": [],
    "accuracy": [],
    "error": [],
    "precision": [],
    "recall": [],
    "f1 score": [],
}
data["calculation"].append("manual")
data["accuracy"].append((TP + TN) / (TN + FP + FN + TP))
data["error"].append((FP + FN) / (TN + FP + FN + TP))
data["precision"].append(precision)  # TP / (TP + FP)
data["recall"].append(recall)  # TP / (TP + FN)
data["f1 score"].append(f1_score)  # (2 * precision * recall) / (precision + recall)
df = pd.DataFrame(data)
df

- `scikit-learn` has functions for [these metrics](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics).

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

data["accuracy"].append(accuracy_score(y_valid, pipe_lr.predict(X_valid)))
data["error"].append(1 - accuracy_score(y_valid, pipe_lr.predict(X_valid)))
data["precision"].append(
    precision_score(y_valid, pipe_lr.predict(X_valid), zero_division=1)
)
data["recall"].append(recall_score(y_valid, pipe_lr.predict(X_valid)))
data["f1 score"].append(f1_score(y_valid, pipe_lr.predict(X_valid)))
data["calculation"].append("sklearn")
df = pd.DataFrame(data)
df.set_index(["calculation"])

The scores match, of course!

## `classification_report` function

There is a convenient function called `classification_report` in `sklearn` which gives this info. 

In [ ]:
pipe_lr.classes_

In [ ]:
from sklearn.metrics import classification_report

print(
    classification_report(
        y_valid, pipe_lr.predict(X_valid), target_names=["non-fraud", "fraud"]
    )
)

⚠️⚠️⚠️ The "rounding to 2 precision points" has resulted in some 1.00 in the above table.

----------
**[study for yourself at home]**

### Macro average

- You give equal importance to all classes and average over all classes.  
- For instance, in the example above, recall for non-fraud is 1.0 and fraud is 0.63, and so macro average is 0.81. 
- More relevant in case of multi-class problems.    

### Weighted average

- Weighted by the number of samples in each class. 
- Divide by the total number of samples. 

Which one is relevant when depends upon whether you think each class should have the same weight or each sample should have the same weight. 

### Toy example

In [ ]:
from sklearn.metrics import classification_report
y_true = [0, 1, 0, 1, 0]
y_pred = [0, 0, 0, 1, 0]
target_names = ['class 0', 'class 1']
print(classification_report(y_true, y_pred, target_names=target_names))

- weighted average is weighted by the proportion of examples in a particular class. So for the toy example above:
- weighted_average precision: 3/5 * 0.75 + 2/5 * 1.00 = 0.85
- weighted_average recall: 3/5 * 1.00 + 2/5 * 0.5 = 0.80
- weighted_average f1-score: 3/5 * 0.86 + 2/5 * 0.67 = 0.78

- macro average gives equal weight to both classes. So for the toy example above:
- macro average precision: 0.5 * 0.75 + 0.5 * 1.00 =0. 875
- macro average recall: 0.5 * 1.00 + 0.5 * 0.5 =0. 75
- macro average f1-score: 0.5 * 0.75 + 0.5 * 1.00 =0.765

**[study for yourself at home]**

----------


## Interim summary 

- Accuracy is misleading when you have class imbalance. 
- A confusion matrix provides a way to break down errors made by our model. 
- We looked at three metrics based on confusion matrix: 
    - precision, recall, f1-score. 

- Note that in **Binary Classification**, what you consider **"positive"** (e.g. fraud in our case) is important when calculating precision, recall, and f1-score. 
  - If you flip what is considered positive or negative, we'll end up with different TP, FP, TN, FN, and hence different precision, recall, and f1-scores. 


<br><br><br><br><br><br><br><br>

### Evalution metrics overview  
There is a lot of terminology here. 

<img src='../img/evaluation-metrics.png' width="1000" height="1000" />

## `cross_validate` with different metrics

We can pass different evaluation metrics with `scoring` argument of `cross_validate`.
- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html

In [ ]:
pipe = make_pipeline(StandardScaler(), LogisticRegression())

# ------ New code -----------------
scores = cross_validate(
    pipe, X_train_big, y_train_big, return_train_score=True, scoring=["accuracy", "f1", "recall", "precision"]
)
# ---------------------------------

pd.DataFrame(scores)

- You can also create [your own scoring function](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html) and pass it to `cross_validate`. 

<br><br>

## ❓❓ Questions for you

### (iClicker) Exercise 9.1 

**iClicker cloud join link: https://join.iclicker.com/EMMJ**

**Select all of the following statements which are TRUE.**

- (A) In medical diagnosis, false positives are likely to be more damaging than false negatives (assume "positive" means the person has a disease, "negative" means they are healthy).
- (B) In spam classification, false positives are more damaging than false negatives (assume "positive" means the email is spam, "negative" means it's not).
- (C) If method A gets a higher accuracy than method B, that means its precision is also higher.
- (D) If method A gets a higher accuracy than method B, that means its recall is also higher.

<!-- <img src='img_aa/medical_fp_vs_fn.png' width="700" /> -->

<br><br><br><br>

<!-- Method A - higher accuracy but lower precision

| Negative | Positive
| -------- |:-------------:|
| 90      | 5|
| 5      | 0|

Method B - lower accuracy but higher precision

| Negative | Positive
| -------- |:-------------:|
| 80      | 15|
| 0      | 5|


 -->

<br><br><br><br>

# Precision-Recall curve / ROC curve

- Confusion matrix provides a detailed break down of the errors made by the model. 
- But when creating a confusion matrix, we are using "hard" predictions. 
- Most classifiers in `scikit-learn` provide **`predict_proba` method (or `decision_function`) which provides degree of certainty about predictions** by the classifier. 
- Can we explore the degree of uncertainty to understand and improve the model performance? 

Let's revisit the classification report on our fraud detection example. 

In [ ]:
pipe_lr = make_pipeline(StandardScaler(), LogisticRegression())
pipe_lr.fit(X_train, y_train);

In [ ]:
y_pred = pipe_lr.predict(X_valid)
print(classification_report(y_valid, y_pred, target_names=["non-fraud", "fraud"]))

**By default, predictions use the threshold of 0.5. If `predict_proba` > 0.5, predict "fraud" else predict "non-fraud".**

In [ ]:
# ------- New Code -------------------
y_pred = pipe_lr.predict_proba(X_valid)[:, 1] > 0.50
# ------------------------------------
print(classification_report(y_valid, y_pred, target_names=["non-fraud", "fraud"]))

- Suppose for your business it is more costly to miss fraudulent transactions and you want to achieve a recall of at least 75% for the "fraud" class. 
- One way to do this is by changing the threshold of `predict_proba`.
    - `predict` returns 1 when `predict_proba`'s probabilities are above 0.5 for the "fraud" class.

**Key idea: what if we threshold the probability at a smaller value so that we identify more examples as "fraud" examples?** 

Let's lower the threshold to **0.1**   
In other words, predict the examples as "fraud" if `predict_proba` > 0.1.  

In [ ]:
y_pred_lower_threshold = pipe_lr.predict_proba(X_valid)[:, 1] > 0.1

In [ ]:
print(classification_report(y_valid, y_pred_lower_threshold, target_names=["non-fraud", "fraud"]))

## Precision/Recall tradeoff 

- But there is a trade-off between precision and recall. 
- If you identify more things as "fraud", recall is going to increase but there are likely to be more false positives. 

Let's sweep through different thresholds. 

In [ ]:
thresholds = np.arange(0.0, 1.0, 0.1)
thresholds

In [ ]:
def f(threshold):
    preds = pipe_lr.predict_proba(X_valid)[:, 1] > threshold    
    print("Threshold: ", np.round(threshold,4))
    print("Precision: ", np.round(precision_score(y_valid, preds),4))
    print("Recall: ", np.round(recall_score(y_valid, preds), 4))    
    print("f1 score: ", np.round(f1_score(y_valid, preds), 4))        

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact, interactive

interactive(
    f,
    threshold=widgets.FloatSlider(min=0, max=0.9, step=0.1, value=0.5),
)

Let's view P and R for different thresholds as a dataframe. 

In [ ]:
pr_dict = {"threshold": [], "precision": [], "recall": [], "f1 score": []}
for threshold in thresholds:
    preds = pipe_lr.predict_proba(X_valid)[:, 1] > threshold
    pr_dict["threshold"].append(threshold)
    pr_dict["precision"].append(precision_score(y_valid, preds))
    pr_dict["recall"].append(recall_score(y_valid, preds))
    pr_dict["f1 score"].append(f1_score(y_valid, preds))
pd.DataFrame(pr_dict)    

### Decreasing the threshold

- Decreasing the threshold means a lower bar for predicting fraud. 
    - You are willing to risk more false positives in exchange of more true positives. 
    - recall would either stay the same or go up and precision is likely to go down
    - occasionally, precision may increase if all the new examples after decreasing the threshold are TPs. 

### Increasing the threshold

- Increasing the threshold means a higher bar for predicting fraud. 
    - recall would go down or stay the same but precision is likely to go up 
    - occasionally, precision may go down as the denominator for precision is TP+FP.    

## Operating point 

How to decide which threshold to use?
- It is decided by the **operating point** of the business.
- Setting a requirement on a classifier (e.g., recall of >= 0.75) is called setting the **operating point**. 
- It's usually driven by **business goals** and is useful to make **performance guarantees to customers**. 

## Precision-Recall (PR) curve

Often, when developing a model, it's not always clear what the operating point will be and to understand the the model better, it's informative to look at all possible thresholds and corresponding trade-offs of precision and recall in a plot.  


In [ ]:
from sklearn.metrics import precision_recall_curve

precision, recall, thresholds = precision_recall_curve(
    y_valid, pipe_lr.predict_proba(X_valid)[:, 1]
)
plt.plot(precision, recall, label="logistic regression: PR curve")
plt.xlabel("Precision")
plt.ylabel("Recall")
plt.plot(
    precision_score(y_valid, pipe_lr.predict(X_valid)),
    recall_score(y_valid, pipe_lr.predict(X_valid)),
    "or",
    markersize=10,
    label="threshold 0.5",
)
plt.legend(loc="best");

- Each point in the curve corresponds to a possible threshold of the `predict_proba` output. 
- We can achieve a recall of 0.8 at a precision of 0.4. 
- The red dot marks the point corresponding to the threshold 0.5.
- **The top-most right-most point in this PR diagram would be a perfect classifier (precision = recall = 1).**

- The threshold is not shown here, but it's going from 0 (upper-left) to 1 (lower right).
- At a threshold of 0 (upper left), we are classifying everything  as "fraud".
- Raising the threshold increases the precision but at the expense of lowering the recall. 
- At the extreme right, where the threshold is 1, we get into the situation where all the examples classified as "fraud" are actually "fraud"; we have no false positives. 
- Here we have a high precision but lower recall. 
- Usually the goal is to **keep recall high as precision goes up**. 

### A few comments on PR curve

- Different classifiers might work well in different parts of the curve, i.e., at different operating points.   
- We can compare PR curves of different classifiers to understand these differences. 

### Average Precision (AP) score 

- Often it's useful to have one number summarizing the PR plot (e.g., in hyperparameter optimization)
- One way to do this is by **computing the area under the PR curve**. 
- This is called **average precision** (AP score)
- AP score has a value between 0 (worst) and 1 (best). 

In [ ]:
from sklearn.metrics import average_precision_score
y_predict = pipe_lr.predict_proba(X_valid)[:, 1]

#  -------- New  Code --------------
ap_lr = average_precision_score(y_valid, y_predict)
# ----------------------------------
print("Average precision of logistic regression: {:.3f}".format(ap_lr))

You can also use the following handy function of `sklearn` to get the PR curve and the corresponding AP score. 

In [ ]:
from sklearn.metrics import PrecisionRecallDisplay

#  -------- New  Code --------------
PrecisionRecallDisplay.from_estimator(pipe_lr, X_valid, y_valid);
# ----------------------------------

### AP vs. F1-score

It is very important to note this distinction:

- F1 score is for a given threshold and measures the quality of `predict`.
- AP score is a summary across thresholds and measures the quality of `predict_proba`.


<br>

Remember to pick the desired threshold based on the results on the validation set and **NOT** on the test set.

<br><br><br><br>

<br><br><br>
For classification problems with **imbalanced classes**, using AP score is more meaningful than using accuracy. 
<br><br><br>

### A few comments on PR curve

- Different classifiers might work well in different parts of the curve, i.e., at different operating points.   
- We can compare PR curves of different classifiers to understand these differences. 
- Let's create PR curves for SVC and Logistic Regression. 

In [ ]:
from sklearn.svm import SVC
pipe_svc = make_pipeline(StandardScaler(), SVC())
pipe_svc.fit(X_train, y_train)

In [ ]:
pipe_lr = make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000))
pipe_lr.fit(X_train, y_train)

How to get precision and recall for different thresholds? 
- Use the function `precision_recall_curve`

In [ ]:
precision_lr, recall_lr, thresholds_lr = precision_recall_curve(
    y_valid, pipe_lr.predict_proba(X_valid)[:, 1]
)

In [ ]:
thresholds_lr.shape, precision_lr.shape, recall_lr.shape

-------------
How many thresholds? 
- The number of **unique `predict_proba` scores** in our dataset. 



In [ ]:
len(np.unique(pipe_lr.predict_proba(X_valid)[:, 1]))

- For each threshold, precision and recall is calculated.  
- The last precision and recall values are 1. and 0. respectively and do not have a corresponding threshold. 

-------------

About SVC and confidence of predictions:
- SVC doesn't have `predict_proba`. Instead it has something called `decision_function`. 
- The index of the threshold that is closest to 0 of decision function is the default threshold in SVC. 

In [ ]:
precision_lr, recall_lr, thresholds_lr = precision_recall_curve(
    y_valid, pipe_lr.predict_proba(X_valid)[:, 1]
)
precision_svc, recall_svc, thresholds_svc = precision_recall_curve(
    y_valid, pipe_svc.decision_function(X_valid)
)

In [ ]:
# Finding the threhold indices which are close to zero (default threshold) in LR and SVC
close_default_lr = np.argmin(np.abs(thresholds_lr - 0.5))
close_zero_svm = np.argmin(np.abs(thresholds_svc))

# A bit of plotting
plt.plot(precision_svc, recall_svc, label="svc")
plt.plot(precision_lr, recall_lr, label="logistic regression")
plt.plot(
    precision_svc[close_zero_svm],
    recall_svc[close_zero_svm],
    "o",
    markersize=10,
    label="default threshold svc",
    c="b",
)
plt.plot(
    precision_lr[close_default_lr],
    recall_lr[close_default_lr],
    "*",
    markersize=10,
    label="default threshold logistic regression",
    c="r",
)

plt.xlabel("Precision")
plt.ylabel("Recall")
plt.legend(loc="best");

-----------
**[Study on your own]**

In [ ]:
svc_preds = pipe_svc.predict(X_valid)
lr_preds = pipe_lr.predict(X_valid)

In [ ]:
print("f1_score of logistic regression: {:.3f}".format(f1_score(y_valid, lr_preds)))
print("f1_score of svc: {:.3f}".format(f1_score(y_valid, svc_preds)))

In [ ]:
ap_lr = average_precision_score(y_valid, pipe_lr.predict_proba(X_valid)[:, 1])
ap_svc = average_precision_score(y_valid, pipe_svc.decision_function(X_valid))

In [ ]:
print("Average precision of logistic regression: {:.3f}".format(ap_lr))
print("Average precision of SVC: {:.3f}".format(ap_svc))

- Comparing the precision-recall curves provide us a detail insight compared to f1 score.
- For example, F1 scores for SVC and logistic regressions are pretty similar. In fact, f1 score of logistic regression is a tiny bit better. 
- But when we look at the PR curve, we see that SVC is doing better than logistic regression for most of the other thresholds. 

**[end of Study on your own]**

-----------


<br><br>

### Receiver Operating Characteristic (ROC) curve 

- Another commonly used tool to analyze the behavior of classifiers at different thresholds.  
- Similar to PR curve, it considers all possible thresholds for a given classifier given by `predict_proba` 
  - but instead of precision and recall it plots **false positive rate (FPR)** and **true positive rate (TPR or recall)**.
$$ TPR = \text{recall} = \frac{TP}{TP + FN}$$

$$FPR  = \frac{FP}{FP + TN}$$


In [ ]:
from sklearn.metrics import roc_curve

fpr, tpr, thresholds = roc_curve(y_valid, pipe_lr.predict_proba(X_valid)[:, 1])
plt.plot(fpr, tpr, label="ROC Curve")
plt.xlabel("FPR")
plt.ylabel("TPR (recall)")

default_threshold = np.argmin(np.abs(thresholds - 0.5))

plt.plot(
    fpr[default_threshold],
    tpr[default_threshold],
    "or",
    markersize=10,
    label="threshold 0.5",
)
plt.legend(loc="best");

- The **ideal** curve is close to the **top-left**
    - Ideally, you want a classifier with high recall while keeping low false positive rate.  
- The red dot corresponds to the threshold of 0.5, which is used by predict.
- **In this example**, we see that compared to the default threshold, **we can achieve a better recall of around 0.8 without increasing FPR**. 

-----------
**[Study on your own]**

Let's compare ROC curve of different classifiers. 

In [ ]:
fpr_lr, tpr_lr, thresholds_lr = roc_curve(y_valid, pipe_lr.predict_proba(X_valid)[:, 1])

fpr_svc, tpr_svc, thresholds_svc = roc_curve(
    y_valid, pipe_svc.decision_function(X_valid)
)

In [ ]:
close_default_lr = np.argmin(np.abs(thresholds_lr - 0.5))
close_zero_svm = np.argmin(np.abs(thresholds_svc))

In [ ]:
plt.plot(fpr_svc, tpr_svc, label="svc")
plt.plot(fpr_lr, tpr_lr, label="logistic regression")
plt.plot(
    fpr_svc[close_zero_svm],
    tpr_svc[close_zero_svm],
    "o",
    markersize=10,
    label="default threshold svc",
    c="b",
)
plt.plot(
    fpr_lr[close_default_lr],
    tpr_lr[close_default_lr],
    "*",
    markersize=10,
    label="default threshold logistic regression",
    c="r",
)

plt.xlabel("False positive rate")
plt.ylabel("True positive rate (Recall)")
plt.legend(loc="best");

**[end of Study on your own]**

-----------


### Area under the curve (AUC)

**AUC = Area under the ROC curve**

AUC provides a single meaningful number for the model performance. 

In [ ]:
from sklearn.metrics import roc_auc_score

roc_lr = roc_auc_score(y_valid, pipe_lr.predict_proba(X_valid)[:, 1])
roc_svc = roc_auc_score(y_valid, pipe_svc.decision_function(X_valid))
print("AUC for LR: {:.3f}".format(roc_lr))
print("AUC for SVC: {:.3f}".format(roc_svc))


- **AUC of 0.5 means random chance**. 
  - When AUC is approximately 0.5, the model has no discrimination capacity to distinguish between positive class and negative class.

AUC can be interpreted as evaluating the **ranking** of positive examples.
- What's the probability that a randomly picked positive point has a higher score according to the classifier than a randomly picked point from the negative class. 
- AUC of 1.0 means **all positive points have a higher score than all negative points**. 

Similar to `PrecisionRecallCurveDisplay`, there is a `RocCurveDisplay` function in sklearn. 

In [ ]:
from sklearn.metrics import RocCurveDisplay

RocCurveDisplay.from_estimator(pipe_lr, X_valid, y_valid);

### Let's look at all the scores at once

In [ ]:
scoring = ["accuracy", "f1", "recall", "precision", "roc_auc", "average_precision"]
pipe = make_pipeline(StandardScaler(), LogisticRegression())
scores = cross_validate(pipe, X_train_big, y_train_big, scoring=scoring)
pd.DataFrame(scores).mean()

<br><br><br><br><br>
Check out [these visualization](https://github.com/dariyasydykova/open_projects/tree/master/ROC_animation) on ROC and AUC.  
<br><br><br><br><br>

## Summary
- Precision-Recall (PR) Curve --> AP 
- Receiver Operating Characteristic (ROC) curve --> AUC
- ROC curves should be used when there are roughly equal numbers of observations for each class.
- Precision-Recall curves should be used when there is a moderate to large class imbalance.

Learn more here: https://machinelearningmastery.com/roc-curves-and-precision-recall-curves-for-classification-in-python/

<br><br><br><br>

# Dealing with class imbalance

## Class imbalance in training sets

- This typically refers to having many more examples of one class than another in one's training set.
- Real world data is often imbalanced. 
    - Our Credit Card Fraud dataset is imbalanced.
    - Ad clicking data is usually drastically imbalanced. (Only around ~0.01% ads are clicked.)
    - Spam classification datasets are also usually imbalanced.

## Addressing class imbalance
A very important question to ask yourself: "**Why** do I have a class imbalance?"

- Is it because one class is **naturally more rare** than the other?
- Is it because of my **data collection** methods?
  
In some cases, it may be fine to just ignore the class imbalance.

## Which type of error is more important? 

- False positives (FPs) and false negatives (FNs) have quite different real-world consequences. 
- In PR curve and ROC curve, we saw how changing the prediction threshold can change FPs and FNs. 
- We can then pick the threshold that's appropriate for our problem. 
- Example: if we want high recall, we may use a lower threshold (e.g., a threshold of 0.1). We'll then catch more fraudulent transactions. 

**Reminder**

In [ ]:
pipe_lr = make_pipeline(StandardScaler(), LogisticRegression())
pipe_lr.fit(X_train, y_train)
y_pred = pipe_lr.predict(X_valid)
print(classification_report(y_valid, y_pred, target_names=["non-fraud", "fraud"]))

And changing threshold to 0.10 :

In [ ]:
y_pred = pipe_lr.predict_proba(X_valid)[:, 1] > 0.10
print(classification_report(y_valid, y_pred, target_names=["non-fraud", "fraud"]))

## Handling imbalance

Can we change the model itself rather than changing the threshold so that it takes into account the errors that are important to us?

There are two common approaches for this: 
- **Changing the data (optional)** (not covered here)
   - Undersampling
   - Oversampling 
       - Random oversampling
       - SMOTE 
- **Changing the training procedure** 
    - `class_weight`

### Changing the training procedure 

- Most of `sklearn` classifiers have a parameter called `class_weight`.
- This allows you to specify that one class is more important than another.
  - For example, maybe a false negative is 10x more problematic than a false positive. 

### Example: `class_weight` parameter of `sklearn LogisticRegression` 

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

> class sklearn.linear_model.LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, **class_weight=None**, random_state=None, solver='lbfgs', max_iter=100, multi_class='auto', verbose=0, warm_start=False, n_jobs=None, l1_ratio=None)

> class_weight: dict or 'balanced', default=None

> Weights associated with classes in the form {class_label: weight}. If not given, all classes are supposed to have weight one. 

In [ ]:
url = "https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html"
HTML("<iframe src=%s width=1000 height=650></iframe>" % url)

**Let's train with default parameters:

In [ ]:
ConfusionMatrixDisplay.from_estimator(
    pipe_lr,
    X_valid,
    y_valid,
    display_labels=["Non fraud", "fraud"],
    values_format="d",
    cmap=plt.cm.Blues,
);

**Let's set "fraud" class a weight of 10.**

In [ ]:
pipe_lr_weight = make_pipeline(
    StandardScaler(), 
    LogisticRegression(
        max_iter=500, 
        class_weight={0:1, 1: 10}  # ---> notice the weights
    )
)
pipe_lr_weight.fit(X_train, y_train)
ConfusionMatrixDisplay.from_estimator(
    pipe_lr_weight,
    X_valid,
    y_valid,
    display_labels=["Non fraud", "fraud"],
    values_format="d",
    cmap=plt.cm.Blues,
);

- Notice we've **reduced false negatives** and predicted more Fraud this time.
- This was equivalent to saying give 10x more "importance" to fraud class. 
- Note that as a consequence we are also **increasing false positives**.    

### `class_weight="balanced"`
- A useful setting is `class_weight="balanced"`.
- This sets the weights so that the classes are "equal".

> class_weight: dict, ‘balanced’ or None

> The “balanced” mode uses the values of y to automatically **adjust weights inversely proportional** to class frequencies in the input data as n_samples / (n_classes * np.bincount(y)).

> sklearn.utils.class_weight.compute_class_weight(class_weight, classes, y)

In [ ]:
pipe_lr_balanced = make_pipeline(
    StandardScaler(), LogisticRegression(max_iter=500, class_weight="balanced")
)
pipe_lr_balanced.fit(X_train, y_train)
ConfusionMatrixDisplay.from_estimator(
    pipe_lr_balanced,
    X_valid,
    y_valid,
    display_labels=["Non fraud", "fraud"],
    values_format="d",
    cmap=plt.cm.Blues,
);

We have reduced false negatives but we have many more false positives now ...

## Are we doing better with `class_weight="balanced"`?

In [ ]:
comp_dict = {}
pipe_lr = make_pipeline(StandardScaler(), LogisticRegression(max_iter=500))
scoring = ["accuracy", "f1", "recall", "precision", "roc_auc", "average_precision"]
orig_scores = cross_validate(pipe_lr, X_train_big, y_train_big, scoring=scoring)

In [ ]:
pipe_lr_balanced = make_pipeline(
    StandardScaler(), LogisticRegression(max_iter=500, class_weight="balanced")
)
scoring = ["accuracy", "f1", "recall", "precision", "roc_auc", "average_precision"]
bal_scores = cross_validate(pipe_lr_balanced, X_train_big, y_train_big, scoring=scoring)
comp_dict = {
    "Original": pd.DataFrame(orig_scores).mean().tolist(),
    "class_weight='balanced'": pd.DataFrame(bal_scores).mean().tolist(),
}

In [ ]:
pd.DataFrame(comp_dict, index=bal_scores.keys())

- Recall is much better but precision has dropped a lot; we have many false positives. 
- You could also optimize `class_weight` using hyperparameter optimization for your specific problem. 

- Changing the class weight will **generally reduce accuracy**.
  - The original model was trying to maximize accuracy.
  - Now you're telling it to do something different.
- But that can be fine, **accuracy isn't the only metric that matters**.

-----------
**[Reminder: Covered before]**
**[Re-study on your own]**

### Stratified Sampling when splitting data into Train/Valid/Test

- A similar idea of "balancing" classes can be applied to data splits.
- We have the same option in `train_test_split` with the `stratify` argument. 
- By default it splits the data so that if we have 10% negative examples in total, then each split will have 10% negative examples.

![stratified-sampling](https://cdn.scribbr.com/wp-content/uploads/2020/09/stratified-sample-7.png)

- If you are carrying out cross validation using `cross_validate`, by default it uses [`StratifiedKFold`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html). From the documentation: 

> This cross-validation object is a variation of KFold that returns stratified folds. The folds are made by preserving the percentage of samples for each class.

- In other words, if we have 10% negative examples in total, then each fold will have 10% negative examples.



**[End of Re-study on your own]**

-----------

<br><br><br><br>

## What did we learn today? 

- A number of possible ways to evaluate Classifiers
    - Choose the evaluation metric that makes most sense in your context or which is most common in your discipline  
- Two kinds of binary classification problems 
    - Distinguishing between two classes (e.g., dogs vs. cats)
    - Spotting a class (e.g., spot fraud transaction, spot spam)

- Precision, recall, f1-score are useful when dealing with spotting problems. 
- The thing that we are interested in spotting is considered "positive".   
- Do you need to deal with class imbalance in the given problem? 
- Methods to deal with class imbalance 
    - Changing the training procedure 
        - `class_weight`

### Relevant papers and resources 

- [The Relationship Between Precision-Recall and ROC Curves](https://www.biostat.wisc.edu/~page/rocpr.pdf)
- [Article claiming that PR curve are better than ROC for imbalanced datasets](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0118432)
- [Precision-Recall-Gain Curves: PR Analysis Done Right](https://papers.nips.cc/paper/2015/file/33e8075e9970de0cfea955afd4644bb2-Paper.pdf)
- [ROC animation](https://github.com/dariyasydykova/open_projects/tree/master/ROC_animation)
- [Generalization in Adaptive Data Analysis and Holdout Reuse](https://arxiv.org/pdf/1506.02629.pdf)

![](../img/eva-seeyou.png)